In [ ]:
"""
floorplan_detect.py

Counts:
  • Rooms = interior free-space components not connected to outside
  • Windows = interior components with min(width, height) ≤ 10% of largest interior component’s max dimension
  • Doors = curved arcs (door swings) via RANSAC circle-fitting on wall edges

Tweaks per request:
  - Door radius: 5..85 px
  - RANSAC inlier tol: 4.0 px
  - Min inliers: 22
  - Edge erosion: 2 iters
  - NMS merge tightened: center<14 px & |Δr|≤10 px
  - Inside/outside validation softened; offset = 6 px

Dependencies: pillow, numpy
"""

from __future__ import annotations
import math, random
from collections import deque
from typing import List, Tuple, Dict
import numpy as np
from PIL import Image

# =========================
# ====== IO & Utils =======
# =========================

def load_gray(path: str) -> np.ndarray:
    """Load image as uint8 grayscale [H, W]."""
    return np.array(Image.open(path).convert("L"))

def save_png(path: str, arr_rgb: np.ndarray) -> None:
    Image.fromarray(arr_rgb, mode="RGB").save(path)

def otsu_thresh(gray: np.ndarray) -> int:
    """Simple Otsu threshold to separate dark ink (walls) from background."""
    hist = np.bincount(gray.ravel(), minlength=256).astype(np.float64)
    total = gray.size
    sum_total = np.dot(np.arange(256), hist)
    sumB = 0.0; wB = 0.0; var_max = 0.0; thr = 128
    for t in range(256):
        wB += hist[t]
        if wB == 0: continue
        wF = total - wB
        if wF == 0: break
        sumB += t * hist[t]
        mB = sumB / wB
        mF = (sum_total - sumB) / wF
        var_between = wB * wF * (mB - mF) ** 2
        if var_between > var_max:
            var_max = var_between; thr = t
    return int(thr)

def erode(b: np.ndarray, iterations: int = 1) -> np.ndarray:
    """Binary erosion with a 3×3 structuring element (True=1)."""
    b = b.copy()
    H, W = b.shape
    for _ in range(iterations):
        nb = np.ones_like(b, dtype=bool)
        for dy in (-1, 0, 1):
            for dx in (-1, 0, 1):
                nb &= np.roll(np.roll(b, dy, axis=0), dx, axis=1)
        b = nb
        b[0, :]=b[-1, :]=b[:, 0]=b[:, -1]=False
    return b

def label_components(mask: np.ndarray, connectivity: int = 4) -> Tuple[np.ndarray, int]:
    """Connected components labeling (BFS). Returns (labels, count)."""
    H, W = mask.shape
    labels = np.zeros((H, W), dtype=np.int32)
    cur = 0
    dirs = [(1,0),(-1,0),(0,1),(0,-1)] if connectivity==4 else \
           [(1,0),(-1,0),(0,1),(0,-1),(1,1),(1,-1),(-1,1),(-1,-1)]
    for y in range(H):
        for x in range(W):
            if mask[y, x] and labels[y, x] == 0:
                cur += 1
                q = deque([(y, x)])
                labels[y, x] = cur
                while q:
                    yy, xx = q.popleft()
                    for dy, dx in dirs:
                        ny, nx = yy+dy, xx+dx
                        if 0 <= ny < H and 0 <= nx < W and mask[ny, nx] and labels[ny, nx] == 0:
                            labels[ny, nx] = cur; q.append((ny, nx))
    return labels, cur

def flood_outside(free: np.ndarray) -> np.ndarray:
    """Mark free pixels connected to the border as outside."""
    H, W = free.shape
    outside = np.zeros_like(free, dtype=bool)
    dq = deque()
    for x in range(W):
        if free[0, x]: outside[0, x]=True; dq.append((0,x))
        if free[H-1, x]: outside[H-1, x]=True; dq.append((H-1,x))
    for y in range(H):
        if free[y, 0]: outside[y, 0]=True; dq.append((y,0))
        if free[y, W-1]: outside[y, W-1]=True; dq.append((y, W-1))
    dirs4 = [(1,0),(-1,0),(0,1),(0,-1)]
    while dq:
        y,x = dq.popleft()
        for dy,dx in dirs4:
            ny,nx = y+dy, x+dx
            if 0<=ny<H and 0<=nx<W and free[ny,nx] and not outside[ny,nx]:
                outside[ny,nx]=True; dq.append((ny,nx))
    return outside

def bbox_of_label(labels: np.ndarray, lab: int):
    ys, xs = np.where(labels == lab)
    if len(ys) == 0: return None
    return int(xs.min()), int(ys.min()), int(xs.max()), int(ys.max())

# ===========================================
# ====== Circle / Arc Helpers (RANSAC) ======
# ===========================================

def circle_from_3(p1, p2, p3):
    """Circle through 3 points; returns (cy, cx, r) or None."""
    (y1, x1), (y2, x2), (y3, x3) = p1, p2, p3
    temp = (x2-x1)*(y3-y1) - (y2-y1)*(x3-x1)
    if abs(temp) < 1e-6: return None
    A = np.array([[x2-x1, y2-y1],
                  [x3-x1, y3-y1]], dtype=np.float64)
    B = 0.5*np.array([
        (x2**2 - x1**2) + (y2**2 - y1**2),
        (x3**2 - x1**2) + (y3**2 - y1**2)
    ], dtype=np.float64)
    try:
        cx, cy = np.linalg.solve(A, B)
    except np.linalg.LinAlgError:
        return None
    r = math.hypot(x1 - cx, y1 - cy)
    return (cy, cx, r)

def angle_span(points, center) -> float:
    cy, cx = center
    thetas = np.array([math.atan2(y - cy, x - cx) for (y, x) in points], dtype=np.float64)
    thetas = np.unwrap(thetas)
    return float(thetas.max() - thetas.min())  # radians

def span_ok_deg(deg: float) -> bool:
    return (10 <= deg <= 80) or (100 <= deg <= 120)

# ========================================
# ====== Main Analysis + Visualization ===
# ========================================

def analyze_floorplan(image_path: str,
                      overlay_path: str = "floorplan_detection_overlay.png",
                      window_narrow_ratio: float = 0.10) -> Dict[str, object]:
    """
    Full pipeline:
      1) Walls vs free via Otsu
      2) Interior components → rooms & windows (10% narrowness rule)
      3) Door arcs via RANSAC circle-fitting on edges (tuned)
    """
    gray = load_gray(image_path)
    H, W = gray.shape

    # --- 1) Segment walls vs free ----------------------------------
    thr = otsu_thresh(gray)
    walls = gray < thr
    free  = gray > max(thr + 10, 180)
    unknown = ~(walls | free)
    free |= (unknown & (gray > thr))
    walls |= (unknown & (gray <= thr))

    # --- 2) Rooms & Windows (interior components) -------------------
    outside = flood_outside(free)
    interior = free & (~outside)
    labels_int, n_int = label_components(interior, connectivity=4)

    # narrowness rule for windows
    largest_max_dim = 1
    comps = []
    for lab in range(1, n_int + 1):
        bb = bbox_of_label(labels_int, lab)
        if bb is None: continue
        x0, y0, x1, y1 = bb
        w, h = x1 - x0 + 1, y1 - y0 + 1
        comps.append((lab, x0, y0, x1, y1, w, h))
        largest_max_dim = max(largest_max_dim, max(w, h))

    window_labels = set()
    for lab, x0, y0, x1, y1, w, h in comps:
        if min(w, h) <= window_narrow_ratio * largest_max_dim:
            window_labels.add(lab)

    windows_count = len(window_labels)
    rooms_count   = n_int - windows_count

    # --- 3) Doors via RANSAC arc detection (tuned) ------------------
    # Edge with moderate erosion (2 may erase arc pixels on some scans; try 1)
    walls_er = erode(walls, iterations=1)         # CHANGED: was 2
    edge = walls ^ walls_er
    labels_edge, n_edge = label_components(edge, connectivity=8)

    # Tuned params
    R_MIN, R_MAX      = 5, 85
    RANSAC_ITERS      = 4000        # more tries
    INLIER_TOL_PX     = 4.5         # thicker arcs accepted
    MIN_INLIERS       = 18          # short arcs accepted
    MAX_ARCS_PER_COMP = 3           # NEW: allow multiple arcs in one component

    # Much tighter merging so we don't collapse distinct doors
    NMS_CENTER_PX     = 6           # was 14/8; be conservative
    NMS_DR_PX         = 4

    def span_ok_deg(deg: float) -> bool:
        return (10 <= deg <= 80) or (100 <= deg <= 120)

    def circle_from_3(p1, p2, p3):
        (y1, x1), (y2, x2), (y3, x3) = p1, p2, p3
        temp = (x2-x1)*(y3-y1) - (y2-y1)*(x3-x1)
        if abs(temp) < 1e-6: return None
        A = np.array([[x2-x1, y2-y1],[x3-x1, y3-y1]], dtype=np.float64)
        B = 0.5*np.array([
            (x2**2 - x1**2) + (y2**2 - y1**2),
            (x3**2 - x1**2) + (y3**2 - y1**2)
        ], dtype=np.float64)
        try:
            cx, cy = np.linalg.solve(A, B)
        except np.linalg.LinAlgError:
            return None
        r = math.hypot(x1 - cx, y1 - cy)
        return (cy, cx, r)

    def angle_span(points, center) -> float:
        cy, cx = center
        thetas = np.array([math.atan2(y - cy, x - cx) for (y, x) in points], dtype=np.float64)
        thetas = np.unwrap(thetas)
        return float(thetas.max() - thetas.min())  # radians

    detected = []  # (lab, cy, cx, r, span_deg, inliers)
    for lab in range(1, n_edge + 1):
        ys, xs = np.where(labels_edge == lab)
        if len(ys) < 50:
            continue

        # Allow multiple arcs per component by peeling inliers iteratively
        pts_all = list(zip(ys, xs))
        used = np.zeros(len(pts_all), dtype=bool)
        idxs_all = np.arange(len(pts_all))

        arcs_found = 0
        while arcs_found < MAX_ARCS_PER_COMP:
            avail = idxs_all[~used]
            if len(avail) < 50:
                break

            best = None
            best_inlier_idx = None

            # RANSAC on remaining points only
            for _ in range(RANSAC_ITERS):
                # need 3 random available points
                if len(avail) < 3:
                    break
                i1, i2, i3 = np.random.choice(avail, size=3, replace=False)
                c = circle_from_3(pts_all[i1], pts_all[i2], pts_all[i3])
                if c is None:
                    continue
                cy, cx, r = c
                if not (R_MIN <= r <= R_MAX):
                    continue

                d = np.sqrt((np.array([p[1] for p in pts_all]) - cx)**2 +
                            (np.array([p[0] for p in pts_all]) - cy)**2)
                resid = np.abs(d - r)
                inlier_idx = np.where((resid < INLIER_TOL_PX) & (~used))[0]
                if len(inlier_idx) < MIN_INLIERS:
                    continue

                inlier_pts = [pts_all[i] for i in inlier_idx]
                span_deg = abs(angle_span(inlier_pts, (cy, cx)) * 180.0 / math.pi)
                if not span_ok_deg(span_deg):
                    continue

                if best is None or len(inlier_idx) > len(best_inlier_idx):
                    best = (cy, cx, r, span_deg)
                    best_inlier_idx = inlier_idx

            if best is None:
                break  # no more arcs in this component

            cy, cx, r, span_deg = best
            inlier_pts = [pts_all[i] for i in best_inlier_idx]
            detected.append((lab, cy, cx, r, span_deg, inlier_pts))

            # peel these inliers so we can find a second arc in the same component
            used[best_inlier_idx] = True
            arcs_found += 1

    # Non-max suppression (merge duplicates very conservatively)
    merged = []
    for item in detected:
        lab, cy, cx, r, span_deg, inliers = item
        keep = True
        for j in range(len(merged)):
            _, cy2, cx2, r2, span2, in2 = merged[j]
            if math.hypot(cx - cx2, cy - cy2) < NMS_CENTER_PX and abs(r - r2) <= NMS_DR_PX:
                # keep the stronger fit
                if len(inliers) > len(in2):
                    merged[j] = item
                keep = False
                break
        if keep:
            merged.append(item)

    # Softer geometric validation and sample farther from ink
    def validate_arc_multisample(cy, cx, r, inliers, gray_img) -> bool:
        thetas = np.array([math.atan2(y - cy, x - cx) for (y, x) in inliers])
        thetas = np.unwrap(thetas)
        ok = 0
        for th in np.percentile(thetas, [10, 30, 50, 70, 90]):
            offset = 10  # was 6/8; sample deeper into free-space / wall
            yin  = int(round(cy + (r - offset) * math.sin(th)))
            xin  = int(round(cx + (r - offset) * math.cos(th)))
            yout = int(round(cy + (r + offset) * math.sin(th)))
            xout = int(round(cx + (r + offset) * math.cos(th)))
            if 0 <= yin < H and 0 <= xin < W and 0 <= yout < H and 0 <= xout < W:
                a = gray_img[yin, xin]   # inside/free (should be brighter)
                b = gray_img[yout, xout] # outside/wall (should be darker)
                # very tolerant; your scan shows wide variation
                if (a - b) > 4 and a > 150 and b < 245:
                    ok += 1
        return ok >= 2  # majority of 5 is 3, but allow 2 to avoid over-rejecting

    door_arcs = []
    debug_candidates = []  # (r, span, inliers)
    for (lab, cy, cx, r, span_deg, inliers) in merged:
        debug_candidates.append((r, span_deg, len(inliers)))
        if validate_arc_multisample(cy, cx, r, inliers, gray):
            door_arcs.append((lab, cy, cx, r, span_deg, inliers))

    # Debug prints
    debug_candidates.sort(key=lambda t: t[2], reverse=True)
    print("[DEBUG] Top arc candidates (radius px, span deg, inliers):")
    for k in debug_candidates[:10]:
        print("   r=%.1f, span=%.1f°, inliers=%d" % k)

    doors_count = len(door_arcs)



    door_arcs = []
    debug_candidates = []  # (r, span, inliers)
    for (lab, cy, cx, r, span_deg, inliers) in merged:
        debug_candidates.append((r, span_deg, len(inliers)))
        if validate_arc_multisample(cy, cx, r, inliers, gray):
            door_arcs.append((lab, cy, cx, r, span_deg, inliers))

    # Debug prints to help lock parameters quickly
    debug_candidates.sort(key=lambda t: t[2], reverse=True)
    print("[DEBUG] Top arc candidates (radius px, span deg, inliers):")
    for k in debug_candidates[:10]:
        print("   r=%.1f, span=%.1f°, inliers=%d" % k)

    doors_count = len(door_arcs)

    # --- 4) Overlay for sanity check -------------------------------
    rgb = np.stack([gray, gray, gray], axis=-1).astype(np.uint8)

    # tint rooms/windows
    for lab in range(1, n_int + 1):
        bb = bbox_of_label(labels_int, lab)
        if bb is None: continue
        ys, xs = np.where(labels_int == lab)
        tint = (240, 220, 120) if lab in window_labels else (165, 245, 165)
        r0, g0, b0 = tint
        for y, x in zip(ys, xs):
            R, G, B = rgb[y, x]
            rgb[y, x] = ((R + r0)//2, (G + g0)//2, (B + b0)//2)

    # mark door arcs (cyan)
    for (_, _, _, _, _, inliers) in door_arcs:
        for (y, x) in inliers:
            rgb[y, x] = (0, 170, 255)

    save_png(overlay_path, rgb)

    return {
        "rooms": int(rooms_count),
        "windows": int(windows_count),
        "doors": int(doors_count),
        "overlay_png": overlay_path
    }

# =====================
# ======= Main ========
# =====================

if __name__ == "__main__":
    # set the image path here (PGM/PNG/JPG):
    IMAGE_PATH = "plan.png"   # e.g., "plan.pgm" or full path like r"C:\path\to\plan.png"
    result = analyze_floorplan(IMAGE_PATH, overlay_path="floorplan_detection_overlay.png")
    print(result)
